In [3]:
# load necessary files
import numpy as np
import os
from PIL import Image, ImageOps    # to read the image only
from pylab import plt
import pandas as pd

#Specs
IMAGE_WIDTH  = 1280       # shape[1] represents width.
IMAGE_HEIGHT = 1024       # shape[0] represents height 
REMOVE_N_ROWS  = -240     # -240/None how many last rows in every image to remove  or USE   None for no removal
REMOVE_N_COLUMNS = -480   # -480/None how many last columns in every image to remove or USE None for no removal
IMG_EXT = '.png'          # image extensions in the directory
FOLDER_ABS_PATH = '/Users/bashit.a/Documents/Alzheimer/Dec-15/cam_test'   # images directory location

In [5]:
%matplotlib widget

def tile_images(IMAGE_WIDTH, IMAGE_HEIGHT, FOLDER_ABS_PATH, REMOVE_N_ROWS=None, REMOVE_N_COLUMNS=None, IMG_EXT='png', show_grid=False, return_tiled_files=False):

    rootDir = os.getcwd()     # get current directory
    try:
        f = lambda x, Estep : [int(var[var.find(x)+2 : var.find(x)+4]) for var in Estep] # +2 (add +2(for -,x = 2 indices) index after finding -x ): +4 (x can have maximum 2 digits)

        # change python running directory to the FOLDER_ABS_PATH
        os.chdir(FOLDER_ABS_PATH)

        Timages = sorted([each for each in os.listdir(os.getcwd()) if each.endswith(IMG_EXT)])     # get all files in the current dirctory "endswith"
        Nfiles = np.unique([f[:f.find('-x')] for f in Timages])         # get number of steps/files in the current directory f.find('-x') --> first occurance of '-x' = steps1.2

        Estep = {} ; Files2D = {}; img = {}; img_tile = {}; minmax = {} # declare data holder dictionary

        for idx, Nstep in enumerate(Nfiles):                       
            Estep[idx] = sorted([each for each in os.listdir(os.getcwd()) if each.startswith(Nstep)])      # get all files in the current dirctory "startswith"

            # find minimum and maximum digits after x and y in each file
            xMin, xMax = min(f('-x', Estep[idx])), max(f('-x', Estep[idx]))   
            yMin, yMax = min(f('-y', Estep[idx])), max(f('-y', Estep[idx]))   
            print(f'{Nstep}, x = [{xMin},{xMax}], y = [{yMin},{yMax}]')

            Files2D[Nstep] = np.zeros((yMax-yMin+1, xMax-xMin+1), dtype=object)     # creating zero matrix for plotting with object data type
            img[Nstep] = np.zeros_like(Files2D[Nstep], dtype=object)                # same as Files2D[Nstep] shape

            for file, x, y in zip(Estep[idx], f('-x', Estep[idx]), f('-y', Estep[idx])):
                img_temp =  np.array(ImageOps.grayscale(Image.open(file)))          # read images as grayscale using PIL     

                xIdx, yIdx = x+abs(xMin), y+abs(yMin)      # indices to write values on the grid - abs value is used to make sure indices don't go negetive
                if x==xMax and y==yMin:
                    img[Nstep][yIdx][xIdx] = img_temp[:,:]                                 # right bottom condition
                elif x==xMax:
                    img[Nstep][yIdx][xIdx] = img_temp[:REMOVE_N_ROWS,:]                    # bottom row condition except right bottom
                elif y==yMin:
                    img[Nstep][yIdx][xIdx] = img_temp[:,:REMOVE_N_COLUMNS]                 # right column condition except right bottom     
                else:
                    img[Nstep][yIdx][xIdx] = img_temp[:REMOVE_N_ROWS,:REMOVE_N_COLUMNS]    # crop each image except the above conditions

                Files2D[Nstep][yIdx][xIdx] = file + str(img[Nstep][y+abs(yMin)][x+abs(xMin)].shape)           # Check how grid is formed by file name and shape

            Files2D[Nstep]  = np.flipud(Files2D[Nstep]);          # (0,0) is at the bottom left - File Name Matrix
            img[Nstep]      = np.flipud(img[Nstep])               # (0,0) is at the bottom left - Image Matrix (object type)
            img_tile[Nstep] = np.vstack([np.hstack(list_h) for list_h in img[Nstep]])         # return final image 

        for idx, Nstep in enumerate(Nfiles):           # without seperate loop plotting fails
            f, ax = plt.subplots(num=Nstep)            # figsize = (10,5)
            ax.imshow(img_tile[Nstep], origin='upper', extent=[0,len(img_tile[Nstep][0,:]),0,len(img_tile[Nstep][:,0])], aspect='equal')   # [0, max(columns), 0, max(rows)]

            CROPPED_WIDTH, CROPPED_HEIGHT = IMAGE_WIDTH+int(REMOVE_N_COLUMNS or 0), IMAGE_HEIGHT+int(REMOVE_N_ROWS or 0)   # get the size of cropped image width and height int(value or 0) requires when value is None    
            xticks      = np.hstack([np.arange(0, (xMax - xMin +1)*CROPPED_WIDTH, CROPPED_WIDTH)])
            yticks      = np.hstack([0, np.arange(IMAGE_HEIGHT, len(img_tile[Nstep]), CROPPED_HEIGHT)])

            ax.set(xticks = xticks, xticklabels = np.asarray(xticks/CROPPED_WIDTH , dtype=int) + xMin,
                   yticks = yticks, yticklabels = np.asarray(yticks/CROPPED_HEIGHT, dtype=int) + yMin,
                   xlabel = 'x -->' , ylabel = 'y -->', title = Nstep);                                 # set xtick and ytick labels

            ax.format_coord = lambda x,y: "(X={0:02d} ".format(np.clip(int(x/CROPPED_WIDTH) + xMin, None, xMax)) + \
                        ("Y={0:02d})".format(np.asarray((y-IMAGE_HEIGHT)/CROPPED_HEIGHT, dtype=int) + yMin +1) if y >= IMAGE_HEIGHT else "Y={0:02d})".format(yMin))
        plt.tight_layout()
        ax.grid(color='r', linestyle='-', linewidth=0.4)
        ax.grid(show_grid)
        
        os.chdir(rootDir)      # all operations are done get back to the root directory
        
        pd.set_option('display.max_columns',None)
        return pd.DataFrame(Files2D[Nstep]) if return_tiled_files==True else None
    except:
        os.chdir(rootDir)      # if fails to execute the desired operation get back to the root dirctory
        
tile_images(IMAGE_WIDTH, IMAGE_HEIGHT, FOLDER_ABS_PATH, REMOVE_N_ROWS, REMOVE_N_COLUMNS, IMG_EXT='png', show_grid=True, return_tiled_files=False)

s2-1.0mm, x = [-7,7], y = [-3,3]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …